In [79]:
import itertools
import pandas as pd
import numpy as np
import csv

In [80]:
file_path = './data/household05032024/table02.tbw'
out_path = './data/out05032024/table02.csv'
dic_path = './data/CensusMainShortForm_new.dcf'

f = open(file_path, 'r')


In [81]:
dimension = []
all_values = []
all_codes= []
all_rows = []
values = []
area_names = []
area_codes = []

In [82]:
i=1
notFirst=False
while True:
        
        line = f.readline()
        if not line:
            if notFirst:
                all_values.append(values)
            break
        else:
            if line.find('[Variable]') != -1:
                if notFirst:
                    all_values.append(values)
                name = f.readline()
                name = name.replace("\n", "")
                name =  name[5:]
                dimension.append(name)
                values = []
                notFirst = True
            if line.find('[Value]') != -1:
                label = f.readline()
                label = label.replace("\n", "")
                label = label[6:]
                #print(label)
                values.append(label)
            if line.find('AreaName') == 0: 
                area = line[9:]
                #print(area)
                area_names.append(area.strip())
            if line.find('BreakKey') == 0: 
                code = line[9:]
                code = code.replace(';', '')
                #print(area)
                area_codes.append(code.strip())
            if line.find('Row') == 0: 
                #row = f.readline()
                row = line[4:]
                i = i+1
                arr = [ x.strip() for x in row.split('  ')]
                all_rows += arr
              

               

In [83]:
all_values

[['%AreaName%'],
 ['Total',
  'None/Never Attended',
  'Other (Informal)',
  'Primary',
  'Secondary',
  'Tertiary',
  'Not Stated/DK'],
 ['Total', 'Yes', 'No', 'DK']]

In [84]:
#print(area_codes)              
#all_values[0] = area_codes                
print(dimension)
#print(area_names)
print(all_values)
#print(all_rows)

['SYSTEM_TOTAL', 'EDUC_COMP_HH_HEAD_VS1', 'H19_1_VS1']
[['%AreaName%'], ['Total', 'None/Never Attended', 'Other (Informal)', 'Primary', 'Secondary', 'Tertiary', 'Not Stated/DK'], ['Total', 'Yes', 'No', 'DK']]


In [85]:
d = open(dic_path, 'r', encoding='utf-8')
header =['cod','desc', 'cod_list']
dim_values = []
total = []
a =  np.empty((0, 3))

def getCodes(d, dim):
    d.seek(0)
    values = []
    dim=dim + "\n"
    while True:
        line = d.readline()
        if not line:
            break
        if line.find(dim) != -1:
            line = d.readline()
            while line.find('Value') != -1:
                line = line.replace("\n", "")
                line = line[6:]
                line = line + ';' + dim
                line = line.replace(':', '-')
                item = line.split(';')
                
                values.append(item)
                line = d.readline()
                if line.find('Note=') != -1:
                    line = d.readline()
    return values


for dim in dimension[1:]:
    values = getCodes(d, dim)
    total = ['T', 'Total', dim]
    values.append(total)
    print(values)
    print("\n")
    a = np.concatenate((a, values), axis=0)

[['1', 'None/Never Attended', 'EDUC_COMP_HH_HEAD_VS1\n'], ['2', 'Other (Informal)', 'EDUC_COMP_HH_HEAD_VS1\n'], ['3', 'Primary', 'EDUC_COMP_HH_HEAD_VS1\n'], ['4', 'Secondary', 'EDUC_COMP_HH_HEAD_VS1\n'], ['5', 'Tertiary', 'EDUC_COMP_HH_HEAD_VS1\n'], ['6', 'Not Stated/DK', 'EDUC_COMP_HH_HEAD_VS1\n'], ['T', 'Total', 'EDUC_COMP_HH_HEAD_VS1']]


[['1', 'Yes', 'H19_1_VS1\n'], ['2', 'No', 'H19_1_VS1\n'], ['9', 'DK', 'H19_1_VS1\n'], ['T', 'Total', 'H19_1_VS1']]




In [86]:
a

array([['1', 'None/Never Attended', 'EDUC_COMP_HH_HEAD_VS1\n'],
       ['2', 'Other (Informal)', 'EDUC_COMP_HH_HEAD_VS1\n'],
       ['3', 'Primary', 'EDUC_COMP_HH_HEAD_VS1\n'],
       ['4', 'Secondary', 'EDUC_COMP_HH_HEAD_VS1\n'],
       ['5', 'Tertiary', 'EDUC_COMP_HH_HEAD_VS1\n'],
       ['6', 'Not Stated/DK', 'EDUC_COMP_HH_HEAD_VS1\n'],
       ['T', 'Total', 'EDUC_COMP_HH_HEAD_VS1'],
       ['1', 'Yes', 'H19_1_VS1\n'],
       ['2', 'No', 'H19_1_VS1\n'],
       ['9', 'DK', 'H19_1_VS1\n'],
       ['T', 'Total', 'H19_1_VS1']], dtype='<U32')

In [87]:
area_varname = [dimension[0]] * len(area_codes)


In [88]:
b = np.column_stack((area_codes, area_names, area_varname))


In [89]:
b

array([['', 'Kenya', 'SYSTEM_TOTAL'],
       ['01', 'Mombasa', 'SYSTEM_TOTAL'],
       ['0101', 'Changamwe', 'SYSTEM_TOTAL'],
       ...,
       ['4709', 'Njiru', 'SYSTEM_TOTAL'],
       ['4710', 'Starehe', 'SYSTEM_TOTAL'],
       ['4711', 'Westlands', 'SYSTEM_TOTAL']], dtype='<U22')

In [90]:
c = np.concatenate((b, a), axis=0)

codes = pd.DataFrame(c)

codes.columns =  header

codes

,cod,desc,cod_list
0,,Kenya,SYSTEM_TOTAL
1,01,Mombasa,SYSTEM_TOTAL
2,0101,Changamwe,SYSTEM_TOTAL
3,0102,Jomvu,SYSTEM_TOTAL
4,0103,Kisauni,SYSTEM_TOTAL
...,...,...,...
399,T,Total,EDUC_COMP_HH_HEAD_VS1
400,1,Yes,H19_1_VS1\n
401,2,No,H19_1_VS1\n
402,9,DK,H19_1_VS1\n


In [91]:
codes['cod_list'].replace(regex=True,inplace=True,to_replace=r'\n',value=r'')
codes

,cod,desc,cod_list
0,,Kenya,SYSTEM_TOTAL
1,01,Mombasa,SYSTEM_TOTAL
2,0101,Changamwe,SYSTEM_TOTAL
3,0102,Jomvu,SYSTEM_TOTAL
4,0103,Kisauni,SYSTEM_TOTAL
...,...,...,...
399,T,Total,EDUC_COMP_HH_HEAD_VS1
400,1,Yes,H19_1_VS1
401,2,No,H19_1_VS1
402,9,DK,H19_1_VS1


In [92]:
all_codes = []
all_codes.append(area_codes)

for i, dim in enumerate(dimension[1:]):
    #print(i, dim)
    arr_cod = []
    for arr in all_values[i+1]:

        d =  ''
        result = codes.loc[(codes['cod_list'] == dim) & (codes['desc'] == arr)]
        try:
            d = result['cod'].values[0]
            arr_cod.append(d)
        except:
            pass
    print(len(arr_cod))
    all_codes.append(arr_cod)
        




7
4


In [93]:
result = list(itertools.product(*all_codes))

print(len(result))

11004


In [94]:
df = pd.DataFrame(result)
df.columns = dimension
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11004 entries, 0 to 11003
Data columns (total 3 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   SYSTEM_TOTAL           11004 non-null  object
 1   EDUC_COMP_HH_HEAD_VS1  11004 non-null  object
 2   H19_1_VS1              11004 non-null  object
dtypes: object(3)
memory usage: 258.0+ KB


In [95]:
len(all_rows)


11004

In [96]:
df['VALUES'] = all_rows
df['VALUES'] = df['VALUES'].str.replace(',000000000', '')
df

,SYSTEM_TOTAL,EDUC_COMP_HH_HEAD_VS1,H19_1_VS1,VALUES
0,,T,T,12040701
1,,T,1,5555334
2,,T,2,6460824
3,,T,9,24543
4,,1,T,2130090
...,...,...,...,...
10999,4711,5,9,34
11000,4711,6,T,1488
11001,4711,6,1,19
11002,4711,6,2,1450


In [97]:
codes

,cod,desc,cod_list
0,,Kenya,SYSTEM_TOTAL
1,01,Mombasa,SYSTEM_TOTAL
2,0101,Changamwe,SYSTEM_TOTAL
3,0102,Jomvu,SYSTEM_TOTAL
4,0103,Kisauni,SYSTEM_TOTAL
...,...,...,...
399,T,Total,EDUC_COMP_HH_HEAD_VS1
400,1,Yes,H19_1_VS1
401,2,No,H19_1_VS1
402,9,DK,H19_1_VS1


In [98]:
d = codes.loc[codes['cod_list']=='SYSTEM_TOTAL']
d

,cod,desc,cod_list
0,,Kenya,SYSTEM_TOTAL
1,01,Mombasa,SYSTEM_TOTAL
2,0101,Changamwe,SYSTEM_TOTAL
3,0102,Jomvu,SYSTEM_TOTAL
4,0103,Kisauni,SYSTEM_TOTAL
...,...,...,...
388,4707,Makadara,SYSTEM_TOTAL
389,4708,Mathare,SYSTEM_TOTAL
390,4709,Njiru,SYSTEM_TOTAL
391,4710,Starehe,SYSTEM_TOTAL


In [99]:
for cod in dimension:
    
    cod_col = 'DESC_'+cod
    print(cod)
    d = codes.loc[codes['cod_list']==cod]
    df = pd.merge(df, d, how='inner', left_on=cod, right_on='cod')
    df = df.drop('cod', axis=1)
    df = df.drop('cod_list', axis=1)
    df.rename(columns={'desc': cod_col}, inplace=True) 
df

SYSTEM_TOTAL
EDUC_COMP_HH_HEAD_VS1
H19_1_VS1


,SYSTEM_TOTAL,EDUC_COMP_HH_HEAD_VS1,H19_1_VS1,VALUES,DESC_SYSTEM_TOTAL,DESC_EDUC_COMP_HH_HEAD_VS1,DESC_H19_1_VS1
0,,T,T,12040701,Kenya,Total,Total
1,01,T,T,376212,Mombasa,Total,Total
2,0101,T,T,46437,Changamwe,Total,Total
3,0102,T,T,53209,Jomvu,Total,Total
4,0103,T,T,88128,Kisauni,Total,Total
...,...,...,...,...,...,...,...
10999,4707,6,9,73,Makadara,Not Stated/DK,DK
11000,4708,6,9,6,Mathare,Not Stated/DK,DK
11001,4709,6,9,29,Njiru,Not Stated/DK,DK
11002,4710,6,9,13,Starehe,Not Stated/DK,DK


In [100]:
df.to_csv(out_path, index=True, index_label='id')